<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH17_04_%ED%95%9C%EA%B5%AD%EC%96%B4_BERT%EC%9D%98_%EB%A7%88%EC%8A%A4%ED%81%AC%EB%93%9C_%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.4 MB/s eta 0:00:00


### 1. 마스크드 언어 모델과 토크나이저
-----------------------
한국어는 klue/bert-base는 대표적인 한국어 BERT임

In [2]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

from_pt=True는 해당 모델이 기존에는 텐서플로우가 아니라 파이토치로 학습된 모델이었지만, 이를 텐서플로우에서 사용하겠다는 의미임

In [3]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


### 2. BERT의 입력
-----------------
'축구는 정말 재미있는 [MASK]다' 라는 임의의 문장이 있다고 해보자. 이를 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어모델은 [MASK]의 위치에 해당하는 단어를 예측함


마스크드 언어 모델의 예측결과를 보기 위해 klue/bert-base의 토크나이저를 사용하여 해당 문장을 정수 인코딩해 봄

In [6]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [7]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[   2, 4713, 2259, 3944, 6001, 2259,    4,  809,   18,    3]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [8]:
print(inputs['input_ids'])  # 정수 인코딩 결과과

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [9]:
print(inputs['token_type_ids']) # 세그먼트 인코딩 결과 확인, 한개 문장이므로 모두 0

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [10]:
print(inputs['attention_mask'])  # 패딩 토큰을 구분. 패팅이 없으므로 모두 1

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


### 3. [MASK] 토큰 예측하기
--------
FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어모델의 예측 결과를 정리해서 보여줌

In [11]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [12]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963516354560852,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.02595745585858822,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033913888037205,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924334146082401,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844174280762672,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [13]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.8382413983345032,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.028275731950998306,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.01718936301767826,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989727176725864,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382649324834347,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [14]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012557774782181,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.06124049797654152,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.01748666912317276,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.0161318127065897,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360800549387932,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]